In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import pickle
import string
from dateutil import parser

In [2]:
def get_soup(start_url):
  response = requests.get(start_url)
  page = response.text
  return BeautifulSoup(page, "lxml")

In [23]:
from datetime import date, datetime, timedelta

def perdelta(start, end, delta):
  curr = start
  while curr <end:
    yield curr
    curr += delta

urls = []
for suffix in perdelta(date(1990, 1, 1), date(2012, 12, 31), timedelta(days=7)):
  prefix = 'http://www.billboard.com/charts/hot-100/'
  link = prefix + str(suffix)
  urls.append(link)

In [10]:
urls[:3]

['http://www.billboard.com/charts/hot-100/2012-08-27',
 'http://www.billboard.com/charts/hot-100/2012-09-03',
 'http://www.billboard.com/charts/hot-100/2012-09-10']

In [11]:
def get_artist(soup):
  artist = soup.find_all(class_="chart-row__artist")
  artists = []
  for item in artist:
    item = item.text.strip()
    artists.append(item)
  return artists

In [12]:
def get_track(soup):
    track = soup.find_all(class_="chart-row__song")
    tracks = []
    for item in track:
        item = item.text.strip()
        tracks.append(item)
    return tracks

In [24]:
def get_song(soup):
  songContainer = soup.find_all(class_="o-chart-results-list-row-container")
  tracks = []
  artists = []
  for song in songContainer:
    track = song.find("h3").get_text(strip=True)
    artist = song.find('span', {'class' : 'a-no-trucate'}).get_text(strip=True)
    artists.append(artist)
    tracks.append(track)
  return artists, tracks

In [25]:
frames = []
billboard_dict = {}
for url in urls:
  soup = get_soup(url)
  artist, track = get_song(soup)
  dates = [parser.parse(url.split('/')[5])]*len(artist)
  df = pd.DataFrame({'artist': artist, 'track': track, 'publish_date': dates})
  print(dates)
  frames.append(df)

Output hidden; open in https://colab.research.google.com to view.

In [26]:
df_merge = pd.concat(frames).reset_index(drop=True)

In [27]:
df2 = df_merge.drop_duplicates(['track'], keep='first').reset_index(drop=True)

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
df2.head()

,artist,track,publish_date
0,Phil Collins,Another Day In Paradise,1990-01-01
1,Janet Jackson,Rhythm Nation,1990-01-01
2,Linda Ronstadt (Featuring Aaron Neville),Don't Know Much,1990-01-01
3,Technotronic Featuring Felly,Pump Up The Jam,1990-01-01
4,Taylor Dayne,With Every Beat Of My Heart,1990-01-01


In [30]:
with open('/content/drive/My Drive/Project/billboard_unique.pkl', 'wb') as picklefile:
    pickle.dump(df2, picklefile)

In [31]:
def text_clean(x):
  try:
    x = x.decode('utf-8')
  except:
      None
  return x

df2 = df2.applymap(text_clean)

In [32]:
df2.head()

,artist,track,publish_date
0,Phil Collins,Another Day In Paradise,1990-01-01
1,Janet Jackson,Rhythm Nation,1990-01-01
2,Linda Ronstadt (Featuring Aaron Neville),Don't Know Much,1990-01-01
3,Technotronic Featuring Felly,Pump Up The Jam,1990-01-01
4,Taylor Dayne,With Every Beat Of My Heart,1990-01-01


In [34]:
df2.to_csv("/content/drive/My Drive/Project/HotSongsBillBoard.csv", encoding="utf-8", index = False)